In [1]:
import os
import pandas as pd

In [2]:
os.chdir('/mnt/Storage/home/wangyiman/maternal_loading/2.public_data/0_raw_data')

# GSM ids for SRR.fastq in 0_raw_data

In [3]:
import glob
sra_ls = [x.split('.')[0].split('_')[0] for x in glob.glob('SRR*', recursive=True)]

In [159]:
def fetch_geo_id(srr_id) :
    from Bio import Entrez
    import xml.etree.ElementTree as ET

    # Provide your email address to NCBI for identification purposes
    Entrez.email = "2011419@tongji.edu.cn"

    # Search for the corresponding BioProject IDs in NCBI's SRA database
    handle = Entrez.esearch(db="sra", term=srr_id)
    record = Entrez.read(handle)
    handle.close()

    # Retrieve the list of BioProject ID, GSM ID associated with the SRR ID
    handle = Entrez.esummary(db="sra", id=record["IdList"][0])
    summary = Entrez.read(handle)[0]
    handle.close()
    root = ET.fromstring("<root>" + summary['ExpXml'] + "</root>")
    project_id = root.find("Bioproject").text

    root = ET.fromstring("<root>" + summary['ExpXml'] + "</root>")
    experiment = root.find('Experiment')
    gsm_id = experiment.get('name')

    # Retrieve GSE ID associated with the BioProject IDs
    handle = Entrez.esearch(db="gds", term=project_id)
    record = Entrez.read(handle)
    handle.close()
    handle = Entrez.esummary(db="gds", id=record["IdList"][0])
    summary_record = Entrez.read(handle)
    gse_id = summary_record[0]["Accession"]
    
    return [srr_id, gsm_id, gse_id]


In [165]:
%%time
id_tab = []
for srr_id in sra_ls :
    id_tab.append(fetch_geo_id(srr_id))
    
    

CPU times: user 27.8 s, sys: 788 ms, total: 28.6 s
Wall time: 30min 20s


In [276]:
geo_id_df = pd.DataFrame(id_tab, columns = ['SRR ID', 'GSM ID & Description', 'GSE ID'])
geo_id_df

,SRR ID,GSM ID & Description,GSE ID
0,SRR7942609,GSM3406873: WT 3hpf rep1; Danio rerio; RNA-Seq,GSE120643
1,SRR9962820,GSM4030422: RNAseq_MZezh2mutant_3_3hpf_rep6; D...,GSE119070
2,SRR7813743,GSM3381526: acta2-negative rep3; Danio rerio; ...,GSE119718
3,SRR372803,GSM831519: 5dpf-2,GSE32898
4,SRR8591739,GSM3612311: cLCs_4; Danio rerio; RNA-Seq,GSE126763
...,...,...,...
370,SRR9879913,GSM3996849: 36hpfNCC_rep1; Danio rerio; RNA-Seq,GSE135237
371,SRR372799,GSM831515: 28hpf-2,GSE32898
372,SRR7942614,GSM3406878: WT 6hpf rep2; Danio rerio; RNA-Seq,GSE120643
373,SRR7942605,GSM3406869: WT 0hpf rep1; Danio rerio; RNA-Seq,GSE120643


In [277]:
geo_id_df.to_csv('/mnt/Storage/home/wangyiman/maternal_loading/2.public_data/Supple_tables/tableS1_geo_id_descript.tsv', sep = "\t", index = False)

In [164]:
len(sra_ls)

375

# replicates, SRR ids runned

In [231]:
rep_srr_tab = pd.read_csv('/mnt/Storage/home/wangyiman/maternal_loading/2.public_data/bin/runned2_RNAseqExon.clean.sh', delim_whitespace=True, header = None, usecols = [1,5])
rep_srr_tab['SRR ID'] = [', '.join([y.split('_')[0].split('.')[0] for y in x.split(',')]) for x in rep_srr_tab[5]]
rep_srr_tab.columns = ['Replicate name', 5, 'SRR ID']
# rep_srr_tab = pd.DataFrame.from_dict(rep_srr_tab.drop(5, axis = 1).set_index('Replicate name')['SRR ID'].to_dict(), orient = 'index').reset_index().melt(id_vars=['index'], value_name = 'SRR ID').dropna()
# rep_srr_tab.columns = ['Replicate name', 'Number of Runs', 'SRR ID']
rep_srr_tab = rep_srr_tab.drop(5, axis = 1).drop_duplicates()
rep_srr_tab

,Replicate name,SRR ID
0,256cell_1,SRR1049814
1,24cell_1,SRR372787
2,24cell_2,SRR372788
3,1kcell_1,SRR372789
4,1kcell_2,SRR372790
...,...,...
319,muscle_rep1,SRR6798775
320,muscle_rep2,SRR6798776
321,muscle_rep3,SRR6798777
322,5dpf_beta_rep1,SRR8456906


# replicates used for gene expression matrix

In [213]:
used_sample = pd.concat([
    pd.read_csv('/mnt/Storage/home/wangyiman/maternal_loading/2.public_data/analysis_embryoExon_tissueExon/ovary_earlyEmbryo_sample_exon_readcnt_prepDE_sorted.txt', sep = "\t", header = None, usecols = [0]),
    pd.read_csv('/mnt/Storage/home/wangyiman/maternal_loading/2.public_data/analysis_embryoExon_tissueExon/lateEmbryo_tissue_sample_exon_readcnt_prepDE_sorted.txt', sep = "\t", header = None, usecols = [0]),
]).iloc[6:, :]
    
used_sample.columns = ['Replicate name']
used_sample

,Replicate name
6,oocyte_stage3
7,oocyte_stage4
8,1cell_rep1
9,1cell_rep2
10,1cell_rep3
...,...
203,testis_rep1
204,testis_rep2
205,testis_rep3
206,testis_rep4


# ERR ids

In [243]:
err_erx_tab = pd.read_csv('/mnt/Storage/home/wangyiman/maternal_loading/2.public_data/0_raw_data/PRJEB12982.tsv', 
                          sep = "\t", usecols = ['run_accession', 'experiment_accession', 'sample_accession'])
err_erx_tab

,sample_accession,experiment_accession,run_accession
0,SAMEA3892004,ERX1512938,ERR1442561
1,SAMEA3892005,ERX1512939,ERR1442562
2,SAMEA3892006,ERX1512940,ERR1442563
3,SAMEA3892007,ERX1512941,ERR1442564
4,SAMEA3892008,ERX1512942,ERR1442565
...,...,...,...
355,SAMEA3892105,ERX1513293,ERR1442916
356,SAMEA3892106,ERX1513294,ERR1442917
357,SAMEA3892107,ERX1513295,ERR1442918
358,SAMEA3892109,ERX1513296,ERR1442919


In [253]:
rep_err_tab = rep_srr_tab[[x[:3] == 'ERR' for x in rep_srr_tab['SRR ID']]]
rep_err_tab = pd.concat([rep_err_tab, rep_err_tab['SRR ID'].str.split(', ', expand = True)], axis = 1)
rep_err_tab = rep_err_tab[['Replicate name', 0, 1, 2, 3]].melt(id_vars=['Replicate name'])
rep_err_tab

,Replicate name,variable,value
0,elife_30860_1cell_1,0,ERR1442646
1,elife_30860_1cell_2,0,ERR1442647
2,elife_30860_1cell_3,0,ERR1442648
3,elife_30860_1cell_4,0,ERR1442649
4,elife_30860_1cell_5,0,ERR1442650
...,...,...,...
355,elife_30860_5dpf_1,3,ERR1442866
356,elife_30860_5dpf_2,3,ERR1442867
357,elife_30860_5dpf_3,3,ERR1442868
358,elife_30860_5dpf_4,3,ERR1442869


In [255]:
rep_err_erx_tab = pd.merge(rep_err_tab, err_erx_tab, left_on = 'value', right_on = 'run_accession')[['Replicate name', 'sample_accession', 'experiment_accession', 'run_accession']]
rep_err_erx_tab

,Replicate name,sample_accession,experiment_accession,run_accession
0,elife_30860_1cell_1,SAMEA3892105,ERX1513023,ERR1442646
1,elife_30860_1cell_2,SAMEA3892106,ERX1513024,ERR1442647
2,elife_30860_1cell_3,SAMEA3892107,ERX1513025,ERR1442648
3,elife_30860_1cell_4,SAMEA3892109,ERX1513026,ERR1442649
4,elife_30860_1cell_5,SAMEA3892110,ERX1513027,ERR1442650
...,...,...,...,...
355,elife_30860_5dpf_1,SAMEA3892044,ERX1513243,ERR1442866
356,elife_30860_5dpf_2,SAMEA3892045,ERX1513244,ERR1442867
357,elife_30860_5dpf_3,SAMEA3892046,ERX1513245,ERR1442868
358,elife_30860_5dpf_4,SAMEA3892047,ERX1513246,ERR1442869


In [278]:
rep_err_erx_tab.to_csv('/mnt/Storage/home/wangyiman/maternal_loading/2.public_data/Supple_tables/tableS1_ERR_ids_reps.tsv', sep = "\t", index = False)

# merge relicate names, SRR ids, GSM ids

In [238]:
rep_srr_geo_tab = pd.merge(pd.merge(used_sample, rep_srr_tab, on = 'Replicate name'), geo_id_df, on = "SRR ID").drop_duplicates()
rep_srr_geo_tab

,Replicate name,SRR ID,GSM ID & Description,GSE ID
0,oocyte_stage3,SRR7762356,GSM3359499: oocyte_III_RNA_seq; Danio rerio; R...,GSE114954
1,oocyte_stage4,SRR7762357,GSM3359500: oocyte_IV_RNA_seq; Danio rerio; RN...,GSE114954
2,1cell_rep1,SRR9962799,GSM4030401: RNAseq_Wildtype_0hpf_rep1; Danio r...,GSE119070
3,1cell_rep2,SRR9962800,GSM4030402: RNAseq_Wildtype_0hpf_rep2; Danio r...,GSE119070
4,1cell_rep3,SRR9962801,GSM4030403: RNAseq_Wildtype_0hpf_rep3; Danio r...,GSE119070
...,...,...,...,...
348,ventricle_rep1,SRR9336405,GSM3900726: wt1; Danio rerio; RNA-Seq,GSE133130
350,ventricle_rep2,SRR9336406,GSM3900727: wt2; Danio rerio; RNA-Seq,GSE133130
352,ventricle_rep3,SRR9336407,GSM3900728: wt3; Danio rerio; RNA-Seq,GSE133130
354,ventricle_rep4,SRR9336408,GSM3900729: wt4; Danio rerio; RNA-Seq,GSE133130


In [279]:
rep_srr_geo_tab.to_csv('/mnt/Storage/home/wangyiman/maternal_loading/2.public_data/Supple_tables/tableS1_SRR_ids_reps.tsv', sep = "\t", index = False)

In [239]:
len(used_sample['Replicate name'].unique())

318

In [241]:
rep_srr_geo_tab[rep_srr_geo_tab['Replicate name'] == '1cell_rep3']

,Replicate name,SRR ID,GSM ID & Description,GSE ID
4,1cell_rep3,SRR9962801,GSM4030403: RNAseq_Wildtype_0hpf_rep3; Danio r...,GSE119070


# concat SRR and ERR

In [267]:
err_tab_tmp = rep_err_erx_tab[['Replicate name', 'sample_accession', 'run_accession']]
err_tab_tmp.columns = ['Replicate name', 'Sample accession', 'Run accession']
srr_tab_tmp = rep_srr_geo_tab[['Replicate name', 'SRR ID', 'GSM ID & Description']]
srr_tab_tmp.columns = ['Replicate name', 'Sample accession', 'Run accession']
final_tab = pd.concat([err_tab_tmp, srr_tab_tmp], ignore_index = True)
final_tab['Run accession'] = final_tab['Run accession'].str.split(':', expand = True)[0]
final_tab

,Replicate name,Sample accession,Run accession
0,elife_30860_1cell_1,SAMEA3892105,ERR1442646
1,elife_30860_1cell_2,SAMEA3892106,ERR1442647
2,elife_30860_1cell_3,SAMEA3892107,ERR1442648
3,elife_30860_1cell_4,SAMEA3892109,ERR1442649
4,elife_30860_1cell_5,SAMEA3892110,ERR1442650
...,...,...,...
555,ventricle_rep1,SRR9336405,GSM3900726
556,ventricle_rep2,SRR9336406,GSM3900727
557,ventricle_rep3,SRR9336407,GSM3900728
558,ventricle_rep4,SRR9336408,GSM3900729


In [280]:
final_tab.to_csv('/mnt/Storage/home/wangyiman/maternal_loading/2.public_data/Supple_tables/tableS1_rep_ERRSRR_sample_run.tsv', sep = "\t", index = False)

In [269]:
len(final_tab['Replicate name'].unique())

290

In [271]:
final_tab.drop_duplicates()

,Replicate name,Sample accession,Run accession
0,elife_30860_1cell_1,SAMEA3892105,ERR1442646
1,elife_30860_1cell_2,SAMEA3892106,ERR1442647
2,elife_30860_1cell_3,SAMEA3892107,ERR1442648
3,elife_30860_1cell_4,SAMEA3892109,ERR1442649
4,elife_30860_1cell_5,SAMEA3892110,ERR1442650
...,...,...,...
555,ventricle_rep1,SRR9336405,GSM3900726
556,ventricle_rep2,SRR9336406,GSM3900727
557,ventricle_rep3,SRR9336407,GSM3900728
558,ventricle_rep4,SRR9336408,GSM3900729


In [274]:
# read in .tsv

os.chdir('/mnt/Storage/home/wangyiman/maternal_loading/2.public_data/analysis_embryoExon_tissueExon')
expr_3id_dict = {}
for expression_type in ['TPM'] :
    for finame in [f'ovary_earlyEmbryo_sample_exon_{expression_type}.tsv', f'lateEmbryo_tissue_sample_exon_{expression_type}.tsv'] :
        expr_pd = pd.read_table(finame, header=0, sep="\t", encoding="utf-8", nrows = 1)
        expr_3id_dict[f'{finame[:17]}_{expression_type}'] = expr_pd
        

In [275]:
expr_3id_dict[f'{finame[:17]}_{expression_type}']

,exon_id,24hpf_pgc_rep1,24hpf_pgc_rep2,24hpf_pgc_rep3,24hpf_pgc_rep4,24hpf_rep1,24hpf_rep2,24hpf_rep3,24hpf_rep4,24hpf_rep5,...,ventricle_rep1,ventricle_rep2,ventricle_rep3,ventricle_rep4,ventricle_rep5,testis_rep1,testis_rep2,testis_rep3,testis_rep4,testis_rep5
0,ENSDART00000000004.5.1,0.409856,0.221467,0.405261,0.234795,0.048692,0.242575,0.0,0.104251,0.165486,...,0.597491,0.025104,0.191853,0.552912,0.166538,0.128866,0.085461,0.14207,0.054256,0.025338
